# 初めに

数独を焼きなましで解いてみよう！のこぼれ話です。数独の盤面を受け取り、スコアを返す関数をいくつか作成しました。スコアは同じ行・列・ボックスに同じ数字が複数含まれていると高くなり、焼きなましではそのスコアを低くすることにより最適解を探します。スコアを求める方法をいくつか作り、その時に得た高速化の知見をメモします。なお、スコアを求めるには盤面の更新によるスコアの差分を取るほうが速いため、以下の方法は実際にはあまり使いません。

数独の盤面はMatrixで表現しました。スコアの計算方法は次の通りです。  
$$ score = \sum_{i=1}^{9} [(9 - 列iに含まれる数の種類) + (9 - 行iに含まれる数の種類) + (9 - ボックスiに含まれる数の種類)] $$

# 準備

In [1]:
using Random
using BenchmarkTools

In [2]:
board = Random.rand(1:9, 9, 9)

9×9 Matrix{Int64}:
 5  8  1  9  5  2  6  9  9
 9  4  7  2  1  9  9  2  5
 9  1  8  5  7  9  6  4  9
 9  8  2  9  2  9  4  2  5
 5  3  9  4  7  5  6  6  9
 3  4  9  4  4  1  9  9  3
 6  9  4  4  4  7  2  7  2
 5  8  6  6  8  4  4  3  9
 4  6  8  9  1  5  5  8  4

# 方法1:Set()を使う

まずは定義どおりに実装します。同じ列・行・ボックスに含まれる数字の種類は`Set()`を使うことで数えました。後から分かったことですが、`Set()`の処理にかなり時間がかかります。

In [3]:
function evaluate1(board)
    score = 0
    for i in 1:9
        score += (9 - length(Set(board[i,:]))) #行i
        score += (9 - length(Set(board[:,i]))) #列i
        score += (9 - length(Set(board[(i-1)%3 * 3 + 1 : (i-1)%3 * 3 + 3, div(i-1, 3) * 3 + 1 : div(i-1, 3) * 3 + 3])))
    end
    return score
end

evaluate1 (generic function with 1 method)

In [4]:
evaluate1(board)

87

In [5]:
@benchmark evaluate1(board)

BenchmarkTools.Trial: 10000 samples with 6 evaluations.
 Range (min … max):  5.633 μs … 328.967 μs  ┊ GC (min … max): 0.00% … 96.33%
 Time  (median):     5.800 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   6.511 μs ±  11.122 μs  ┊ GC (mean ± σ):  7.14% ±  4.10%

  ▅█▇▄▄▄▃▁                ▁▁▁                                 ▂
  █████████▇▅▅▅▅▁▄▃▃▃▄▄▅▆██████▇▇▇▇▇▆▇▆▇▇▆▆▆▅▅▅▅▅▅▅▅▁▃▄▄▃▃▅▃▄ █
  5.63 μs      Histogram: log(frequency) by time      10.6 μs <

 Memory estimate: 13.92 KiB, allocs estimate: 135.

In [6]:
print(evaluate1(board))

87

# 方法2:登場した数字の種類を配列で管理する

次に配列を使って登場した数字の種類を数えます。`nums`は要素数9の配列で、初期値はすべて0です。7が登場したら、`nums[7] = 1`とすることにより、登場した数字がどれなのか管理します。配列の要素の型をInt64からInt8にすると9%ほど高速になりました。Bool型を使うともっと速くなるのかな？

In [7]:
function evaluate2(board)
    score = 0
    for i in 1:9
        tmp_h = zeros(Int8, 9)
        tmp_v = zeros(Int8, 9)
        tmp_box = zeros(Int8, 9)
        for j in 1:9
            tmp_h[board[i,j]] = 1
            tmp_v[board[j,i]] = 1
            tmp_box[board[((i-1)%3)*3 + (j-1)%3 + 1, div(i-1, 3)*3 + div(j-1, 3) + 1]] = 1
        end
        score += (27 - sum(tmp_h) - sum(tmp_v) - sum(tmp_box))
    end
    return score
end

evaluate2 (generic function with 1 method)

In [8]:
evaluate2(board)

87

In [9]:
@benchmark evaluate2(board)

BenchmarkTools.Trial: 10000 samples with 10 evaluations.
 Range (min … max):  1.090 μs … 158.390 μs  ┊ GC (min … max): 0.00% … 98.52%
 Time  (median):     1.120 μs               ┊ GC (median):    0.00%
 Time  (mean ± σ):   1.264 μs ±   2.700 μs  ┊ GC (mean ± σ):  3.65% ±  1.70%

  ▄█▆▄▃▁    ▂▁           ▄▄▄▁ ▂▂                              ▂
  ██████▆▆▅▆████▇▄▅▃▁▃▁▃▇████████▆▆███▇▅▆▅▅▅▄▁▄▅▃▄▁▄▄▄▃▄▁▃▄▆▆ █
  1.09 μs      Histogram: log(frequency) by time      2.08 μs <

 Memory estimate: 1.69 KiB, allocs estimate: 27.

# 方法3:方法2の改良版。配列の生成回数を減らす

方法2の改良版です。イメージとしては方法2で9回生成していた配列を1つに繋げたというものです。結構高速になりますね。可読性が怪しくなるのが欠点か。

In [10]:
function evaluate3(board)
    tmp_h = zeros(Int8, 9*9)
    tmp_v = zeros(Int8, 9*9)
    tmp_box = zeros(Int8, 9*9)
    for i in 1:9
        for j in 1:9
            tmp_h[board[i,j] + (i-1)*9] = 1
            tmp_v[board[j,i] + (i-1)*9] = 1
            tmp_box[board[((i-1)%3)*3 + (j-1)%3 + 1, div(i-1, 3)*3 + div(j-1, 3) + 1] + (i-1)*9] = 1
        end
    end
    return (9*9*3 - sum(tmp_h) - sum(tmp_v) - sum(tmp_box))
end

evaluate3 (generic function with 1 method)

In [11]:
evaluate3(board)

87

In [12]:
@benchmark evaluate3(board)

BenchmarkTools.Trial: 10000 samples with 196 evaluations.
 Range (min … max):  467.857 ns …  3.778 μs  ┊ GC (min … max): 0.00% … 76.64%
 Time  (median):     476.020 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   491.158 ns ± 95.457 ns  ┊ GC (mean ± σ):  0.69% ±  3.25%

  ▆█▇▆▂                 ▂▂▁▁▁                                  ▂
  █████▇▇▄▄▄▅▄▆▅▁▃▄▄▆▄▆██████▇▇▇▆▆▅▄▄▅▄▅▁▅▅▆▇▆▆▆▆▅▅▆▄▆▆▄▅▅▄▄▅▅ █
  468 ns        Histogram: log(frequency) by time       734 ns <

 Memory estimate: 432 bytes, allocs estimate: 3.

# 方法4:方法3からさらに配列をまとめる

方法3では3つの配列を使っていましたが、それらを1つに繋げました。ここまでくると可読性は壊滅的ですね。

In [13]:
function evaluate4(board)
    tmp = zeros(Int8, 81*3)
    for i in 1:9
        for j in 1:9
            tmp[board[i,j] + (i-1)*9] = 1
            tmp[board[j,i] + (i-1)*9 + 81] = 1
            tmp[board[((i-1)%3)*3 + (j-1)%3 + 1, div(i-1, 3)*3 + div(j-1, 3) + 1] + (i-1)*9 + 81*2] = 1
        end
    end
    return (9*9*3 - sum(tmp))
end

evaluate4 (generic function with 1 method)

In [14]:
evaluate4(board)

87

In [15]:
@benchmark evaluate4(board)

BenchmarkTools.Trial: 10000 samples with 210 evaluations.
 Range (min … max):  362.381 ns …  3.195 μs  ┊ GC (min … max): 0.00% … 82.59%
 Time  (median):     377.619 ns              ┊ GC (median):    0.00%
 Time  (mean ± σ):   392.406 ns ± 90.475 ns  ┊ GC (mean ± σ):  0.71% ±  3.01%

  ▄▇▆█▆▅▂▁      ▁▄▃▂▂                                          ▂
  █████████▇▇▇▇████████▇▇▇▆▇▇▆▆▆▅▅▆▆▆▇▇▇▇▅▇▆▅▆▆▅▄▅▄▁▅▄▅▄▁▄▄▃▅▅ █
  362 ns        Histogram: log(frequency) by time       644 ns <

 Memory estimate: 304 bytes, allocs estimate: 1.

# まとめ

今回の実験で分かったこと
1. `Set()`は時間がかかり、今回のように限られた範囲の値を扱うなら配列のほうが便利かも
2. 0 or 1　のように取りうる値の範囲が限られている場合、Int64よりInt8などを使う方が速い
3. 配列の生成回数は少なくしよう。（`sum()`に時間がかかっているだけかも？）　　


|方法 |ベンチマーク結果|備考|
|:---:|:------------:  |:---|
|方法1|6.511 μs ±  11.122 μs|Set()を使う|
|方法2|1.264 μs ±   2.700 μs|登場した数字の種類を配列で管理する|
|方法3|491.158 ns ± 95.457 ns|方法2の改良版。配列の生成回数を減らす|
|方法4|392.406 ns ± 90.475 n|方法3からさらに配列をまとめる|